# Sentimental_Analysis + LJP ??

## Idea

- facts에 대한 긍정 / 부정 감정도를 조사하면 target 값에 유의미한 연관성이 있지 않을까?

In [12]:
import pandas as pd
import numpy as np

In [13]:
import os
from pathlib import Path

path = Path(os.getcwd() + '/datasets')

In [14]:
train = pd.read_csv(path / 'train.csv')
test = pd.read_csv(path / 'test.csv')
submission = pd.read_csv(path / 'sample_submission.csv')

## Data Overview

In [22]:
pd.set_option('display.max_colwidth', None)
train.head(5)

,ID,first_party,second_party,facts,first_party_winner
0,TRAIN_0000,Phil A. St. Amant,Herman A. Thompson,"On June 27, 1962, Phil St. Amant, a candidate for public office, made a television speech in Baton Rouge, Louisiana. During this speech, St. Amant accused his political opponent of being a Communist and of being involved in criminal activities with the head of the local Teamsters Union. Finally, St. Amant implicated Herman Thompson, an East Baton Rouge deputy sheriff, in a scheme to move money between the Teamsters Union and St. Amant’s political opponent. \nThompson successfully sued St. Amant for defamation. Louisiana’s First Circuit Court of Appeals reversed, holding that Thompson did not show St. Amant acted with “malice.” Thompson then appealed to the Supreme Court of Louisiana. That court held that, although public figures forfeit some of their First Amendment protection from defamation, St. Amant accused Thompson of a crime with utter disregard of whether the remarks were true. Finally, that court held that the First Amendment protects uninhibited, robust debate, rather than an open season to shoot down the good name of anyone who happens to be a public servant. \n",1
1,TRAIN_0001,Stephen Duncan,Lawrence Owens,"Ramon Nelson was riding his bike when he suffered a lethal blow to the back of his head with a baseball bat. After two eyewitnesses identified Lawrence Owens from an array of photos and then a lineup, he was tried and convicted for Nelson’s death. Because Nelson was carrying cocaine and crack cocaine potentially for distribution, the judge at Owens’ bench trial ruled that Owens was probably also a drug dealer and was trying to “knock [Nelson] off.” Owens was found guilty of first-degree murder and sentenced to 25 years in prison.\nOwens filed a petition for a writ of habeas corpus on the grounds that his constitutional right to due process was violated during the trial. He argued that the eyewitness identification should have been inadmissible based on unreliability and that the judge impermissibly inferred a motive when a motive was not an element of the offense. The district court denied the writ of habeas corpus, and Owens appealed. The U.S. Court of Appeals for the Seventh Circuit reversed the denial and held that the trial judge’s inference about Owens’s motive violated his right to have his guilt adjudicated solely based on the evidence presented at trial.\n",0
2,TRAIN_0002,Billy Joe Magwood,"Tony Patterson, Warden, et al.","An Alabama state court convicted Billy Joe Magwood of murder and sentenced him to death. Subsequently, an Alabama federal district court partially granted Mr. Magwood's petition for federal habeas corpus relief. The court upheld his conviction but instructed the state court to look at mitigating evidence when resentencing Mr. Magwood. Upon resentencing, the state court sentenced Mr. Magwood to death once again. Mr. Magwood filed a second petition for federal habeas corpus relief with the federal district court arguing that a judicial rule was retroactively applied in his case and that he lacked effective counsel at sentencing. The district court granted the petition and vacated Mr. Magwood's death sentence.\nOn appeal, the U.S. Court of Appeals for the Eleventh circuit reversed, holding that prisoners may not raise challenges to an original sentence that could have been raised in an earlier petition. The court also held that Mr. Magwood's counsel was not ineffective because he failed to raise an argument that had already been decided by the state's highest court adverse to his client's position.\n",1
3,TRAIN_0003,Linkletter,Walker,"Victor Linkletter was convicted in state court on evidence illegally obtained by police prior to the Supreme Court decision concerning the Fourth Amendment in Mapp v. Ohio. Mapp applied the exclusionary rule to state criminal proceedings, denying the use of illegally obtained evidence at trial. Linkletter argued for a retrial based on the Mapp decision.\n"

## facts : en->ko 번역하여 내용 확인

In [15]:
from googletrans import Translator
translator = Translator()
translator = Translator(service_urls=['translate.googleapis.com'])
res = translator.translate('안녕하세요.', dest='en');

In [16]:
print(res.text)

hello.


In [17]:
train['first_party_winner'].value_counts()

1    1649
0     829
Name: first_party_winner, dtype: int64

In [18]:
train.shape

(2478, 5)

In [19]:
for i in range(5):
    text = train.iloc[i]['facts']
    res = translator.translate(text, dest='ko');
    print(res.text)
    print("-"*50)

1962년 6월 27일, 공직 후보인 Phil St. Amant는 루이지애나주 배턴루지에서 텔레비전 연설을 했습니다. 이 연설에서 St. Amant는 그의 정적이 공산주의자이며 지역 Teamsters Union의 수장과 함께 범죄 활동에 연루되어 있다고 비난했습니다. 마지막으로 St. Amant는 East Baton Rouge 부 보안관인 Herman Thompson을 Teamsters Union과 St. Amant의 정적 사이에서 자금을 이동시키려는 계획에 연루시켰습니다.
Thompson은 명예 훼손으로 St. Amant를 성공적으로 고소했습니다. 루이지애나의 제1 순회 항소 법원은 Thompson이 St. Amant가 "악의"로 행동했음을 보여주지 않았다고 주장하며 뒤집었습니다. 그런 다음 Thompson은 루이지애나 대법원에 항소했습니다. 그 법원은 공인들이 명예 훼손에 대한 수정헌법 제1조의 일부 보호를 박탈당했지만 St. Amant는 발언이 사실인지 여부를 완전히 무시하고 Thompson을 범죄로 고발했다고 판결했습니다. 마지막으로, 그 법원은 수정헌법 제1조가 우연히 공무원이 된 사람의 명예를 깎아내리기 위한 공개적인 시즌보다는 자유롭고 활발한 토론을 보호한다고 판결했습니다.
--------------------------------------------------
Ramon Nelson은 자전거를 타고 가다가 야구 방망이로 머리 뒤쪽을 치명상을 입었습니다. 두 명의 목격자가 일련의 사진과 라인업에서 Lawrence Owens를 확인한 후 그는 Nelson의 죽음으로 재판을 받고 유죄 판결을 받았습니다. Nelson은 유통 가능성이 있는 코카인과 크랙 코카인을 소지하고 있었기 때문에 Owens의 벤치 재판에서 판사는 Owens가 마약상이기도 했으며 "[Nelson]을 노크"하려 했다고 판결했습니다. Owens는 1급 살인죄로 유죄 판결을 받고 25년형을 선고 받았습니다.
Owens는 재판 중에 적법 절차에 대한 헌법상의 권리가 침해되었다는 이유로 인

## 감정분석 NN 학습시키기

### imdb 데이터 로드

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import imdb

In [9]:
(X_train, y_train), (X_test, y_test) = imdb.load_data()

print('훈련용 리뷰 개수 : {}'.format(len(X_train)))
print('테스트용 리뷰 개수 : {}'.format(len(X_test)))
num_classes = len(set(y_train))
print('카테고리 : {}'.format(num_classes))

훈련용 리뷰 개수 : 25000
테스트용 리뷰 개수 : 25000
카테고리 : 2


In [10]:
print('첫번째 훈련용 리뷰 :',X_train[0])
print('첫번째 훈련용 리뷰의 레이블 :',y_train[0])

첫번째 훈련용 리뷰 : [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 22665, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 21631, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 31050, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
첫번째 훈련용 리뷰의 레이블 : 1


### 감정 분석 Modeling

In [7]:
import re
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

vocab_size = 10000
max_len = 500

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [5]:
### 너무 느려서 유료 GPU 서버에서 학습시킴 -> 모델 export 해옴


# embedding_dim = 100
# hidden_units = 128

# model = Sequential()
# model.add(Embedding(vocab_size, embedding_dim))
# model.add(GRU(hidden_units))
# model.add(Dense(1, activation='sigmoid'))

# es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
# mc = ModelCheckpoint('GRU_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

# model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
# history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=64, validation_split=0.2)

In [8]:
loaded_model = load_model( './gru_model')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

Metal device set to: Apple M1 Pro


2023-06-19 18:55:49.873058: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-06-19 18:55:49.873732: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-06-19 18:55:51.805483: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-06-19 18:55:51.948914: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-06-19 18:55:52.008251: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


782/782 [==============================] - 54s 66ms/step - loss: 0.4372 - acc: 0.8179

 테스트 정확도: 0.8179


In [9]:
word_to_index = imdb.get_word_index()
index_to_word = {}
for key, value in word_to_index.items():
    index_to_word[value+3] = key

In [25]:
def sentiment_predict(new_sentence):
  # 알파벳과 숫자를 제외하고 모두 제거 및 알파벳 소문자화
    new_sentence = re.sub('[^0-9a-zA-Z ]', '', new_sentence).lower()
    encoded = []

    # 띄어쓰기 단위 토큰화 후 정수 인코딩
    for word in new_sentence.split():
        try:
          # 단어 집합의 크기를 10,000으로 제한.
            if word_to_index[word] <= 10000:
                encoded.append(word_to_index[word]+3)
            else:
                # 10,000 이상의 숫자는 <unk> 토큰으로 변환.encoded.append(2)
                encoded.append(2)
        # 단어 집합에 없는 단어는 <unk> 토큰으로 변환.
        except KeyError:
            encoded.append(2)

    pad_sequence = pad_sequences([encoded], maxlen=max_len)
    score = float(loaded_model.predict(pad_sequence)) # 예측
    
    if(score > 0.5):
        return 'pos'
    else:
        return 'neg'

In [26]:
s = train.sample(10)

s['fact_kor'] = s['facts'].apply(lambda x: translator.translate(x, dest='ko').text)
s['sentiment_score'] = s['facts'].apply(lambda x: sentiment_predict(x))

1/1 [==============================] - 0s 62ms/step


In [28]:
s

,ID,first_party,second_party,facts,first_party_winner,fact_kor,sentiment_score
2035,TRAIN_2035,Richardson,United States,"A federal criminal statute, 21 U.S.C. section 848(a), proscribes any person from engaging in ""continuing criminal enterprise (CCE),"" which is defined as involving a violation of federal drug statutes where such a violation was part of a ""continuing series of violations."" Eddie Richardson, who had organized and managed the Chicago street gang called the Undertaker Vice Lords in order to sell drugs, was charge with a CCE violation. At trial, Richardson proposed to instruct the jury that it must unanimously agree not only that he committed some ""continuing series of violations"" but also that the he committed each of the individual ""violations"" necessary to make up that ""continuing series."" In other words, the proposed instruction would have required the jury to unanimously agree on which three acts constituted the alleged series of violations. The judge rejected Richardson's proposal and, instead, instructed the jurors that they must unanimously agree that the defendant committed at least three federal narcotics offenses, but did not have to agree as to the particular offenses. Subsequently, the jury convicted Richardson. The Court of Appeals upheld the trial judge's jury instruction.\n",1,"연방 형법, 21 U.S.C. 섹션 848(a)는 위반이 ""계속되는 일련의 위반""의 일부인 연방 마약 법령 위반과 관련된 것으로 정의되는 ""지속적인 범죄 기업(CCE)""에 관여하는 사람을 금지합니다. 마약을 판매하기 위해 Undertaker Vice Lords라고 불리는 시카고 거리 갱단을 조직하고 관리했던 Eddie Richardson은 CCE 위반 혐의로 기소되었습니다. 재판에서 Richardson은 배심원단에게 자신이 ""계속되는 일련의 위반""을 저질렀을 뿐만 아니라 ""계속되는 일련의 위반""을 구성하는 데 필요한 각 개별 ""위반""을 저질렀다는 데 만장일치로 동의해야 한다고 지시할 것을 제안했습니다. 즉, 제안된 지침은 배심원단이 일련의 위반 혐의를 구성하는 세 가지 행위에 대해 만장일치로 동의하도록 요구했을 것입니다. 판사는 Richardson의 제안을 거부하고 대신 피고가 최소 3건의 연방 마약 범죄를 저질렀다는 데 만장일치로 동의해야 하지만 특정 범죄에 대해서는 동의할 필요가 없다고 배심원들에게 지시했습니다. 그 후 배심원은 Richardson에게 유죄를 선고했습니다. 항소법원은 예심 판사의 배심원단 지시를 지지했다.",pos
856,TRAIN_0856,"Ashbel T. Wall, II, Director of Rhode Island Department of Corrections",Khalil Kholi,"In December 1993, a Rhode Island jury convicted Khalil Kholi on 10 counts of first-degree sexual assault. The charges stemmed from the alleged molestation of his two step-daughters. A judge on the state superior court sentenced Kholi to two consecutive terms of life imprisonment, and the state supreme court affirmed the conviction in February 1996. Kholi did not file a federal writ of habeas corpus at that time. Instead, he filed a motion seeking sentence reduction as a form of post-conviction relief, which was denied. Kholi exhausted his procedural options regarding sentence reduction in 2007, at which time he began his appeal for federal writ of habeas corpus, which was well beyond the Antiterrorism and Effective Death Penalty Act's standard one-year limitation on filing. In September 2009, the U.S. Court of Appeals for the First Circuit reversed and remanded the district court's judgment that a petition for leniency is different from an appeal to correct legal errors and therefore does not result in a tolling of the statute of limitations under AEDPA. A circuit split exists on the issue. The First Circuit's decision was in line with a Tenth Circuit ruling on the same issue, but the Third, Fourth and Eleventh Circuits have previously ruled that a petition for leniency does not toll the statute of limitations under AEDPA.\n",0,"1993년 12월 로드 아일랜드 배심원단은 Khalil Kholi를 10건의 1급 성폭행 혐의로 유죄 판결했습니다. 혐의는 그의 두 의붓 딸에 대한 성추행 혐의에서 비롯되었습니다. 주 고등 법원의 판사는 콜리에게 2년 연속 종신형을 선고했고 주 대법원은 1996년 2월 유죄 판결을 확정했습니다. 콜리는 당시 인신 보호 영장을 연방 영장으로 제출하지 않았습니다. 대신 그는 유죄판결 후 구제의 한 형태로 형량 감면을 요청했지만 거부당했습니다. 콜리는 2007년 형량 감형에 관한 절차적 선택권을 모두 소진했으며, 당시 그는 반테러 및 효과적인 사형법의 제출에 대한 표준 1년 제한을 훨씬 넘어서는 인신 보호 영장에 대한 연방 영장에 대한 항소를 시작했습니다. 2009년 9월, 미국 제1순회항소법원은 관용 청원은 법적 오류 수정에 대한 항소와 다르므로 AEDPA에 따른 소멸 시효에 영향을 미치지 않는다는 지방 법원의 판결을 파기하고 환송했습니다. 문제에 회로 분할이 존재합니다. 제1 순회 법원의 결정은 같은 문제에 대한 제10 순회 판결과 일치했지만, 제3, 제4 및 제11 순회 법원은 이전에 관용 청원이 AEDPA에 따른 시효에 영향을 미치지 않는다고 판결했습니다.",pos
497,TR

## Coclusion

- IMDB 데이터는 구어체 <-> 법률 데이터는 문어체 + 법률 용어 Vocab 차이로 데이터 편중으로 모델이 유용하지 못하다
- 감정 분석은 어울리지 않는 접근인 것으로...